In [1]:
import numpy as np
import pandas as pd
import boto3
import os
import re
import io
import smtplib 
from pretty_html_table import build_table
from datetime import datetime
from email.mime.text import MIMEText
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.header import Header
from pathlib import Path

In [2]:

file_path = "C:/Users/abhishek.sriramoju_s/Desktop/keys.txt"  #File path
try:
    with open(file_path, 'r') as file:  #Opening the file that contains credentials
        content = file.read()  #Reading the file
except FileNotFoundError:  
    print(f"File not found at {file_path}")
    content = ""
    
aws_access_key = None  
aws_secret_key = None
service_name = None
region_name = None

lines = content.split('\n') #Spliting the string on the basis of next line
for line in lines:
    if line.startswith("service_name"):    #Getting the Service Name
        service_name = line.split('=')[1].strip()
    elif line.startswith("region_name"):   #Getting the Region Name
        region_name = line.split('=')[1].strip()
    elif line.startswith("AWS_ACCESS_KEY_ID"): #Getting the Access key
        aws_access_key = line.split('=')[1].strip()
    elif line.startswith("AWS_SECRET_ACCESS_KEY"): #Getting the Secret key
        aws_secret_key = line.split('=')[1].strip()

In [3]:
#Connecting to amazon s3 bucket 
resource = boto3.resource(service_name = service_name, #Service Name
                    region_name = region_name,   #Region
                    aws_access_key_id = aws_access_key, #Access key
                    aws_secret_access_key = aws_secret_key) #Secrect key
resource               

s3.ServiceResource()

In [4]:
bucket_name = 'underarmour1'  #Bucket Name
categories_bucket = resource.Bucket(bucket_name) #Getting Bucket object
categories_files = categories_bucket.objects.filter(Prefix='categories/')  #Getting the objects of category file
category_dates = []  
for obj in categories_files :  #Iterating on category files
    name = obj.key  #Category file name
    if name.endswith('.parquet') :  #Checking the condition weather the file is parquet or not
        values = name.split('_')   #Splitting the file names
        values = values[0].split('/')[1]  #Will get the date in yyyy.mm.dd
        date = list(map(int, values.split('.'))) 
        category_dates.append((name, datetime(date[0], date[1], date[2]))) #Appending datet and file name in category_dates
category_dates.sort(key = lambda x: x[1],reverse = True) #Sorting the list on the basis of date


buffer = io.BytesIO()  #Creating an buffer
object = resource.Object(bucket_name,category_dates[0][0]) #The Latest File
object.download_fileobj(buffer) 
category_file_2 = pd.read_parquet(buffer) #Reading the parquet file

buffer = io.BytesIO()
object = resource.Object(bucket_name,category_dates[1][0])
object.download_fileobj(buffer)

category_file_1 = pd.read_parquet(buffer)

# buffer = io.BytesIO()
# object = resource.Object(bucket_name,category_dates[2][0])
# object.download_fileobj(buffer)

# category_file_0 = pd.read_parquet(buffer)
# category_file_0



product_files = categories_bucket.objects.filter(Prefix='products/')  #Getting the objects of category file
product_dates = []  
for obj in product_files :  #Iterating on category files
    name = obj.key  #Category file name
    if name.endswith('.parquet') :  #Checking the condition weather the file is parquet or not
        values = name.split('_')   #Splitting the file names
        values = values[0].split('/')[1]  #Will get the date in yyyy.mm.dd
        date = list(map(int, values.split('.'))) 
        product_dates.append((name, datetime(date[0], date[1], date[2]))) #Appending datet and file name in category_dates
product_dates.sort(key = lambda x: x[1],reverse = True) #Sorting the list on the basis of date


buffer = io.BytesIO()  #Creating an buffer
object = resource.Object(bucket_name,product_dates[0][0]) #The Latest File
object.download_fileobj(buffer) 
product_file_2 = pd.read_parquet(buffer) #Reading the parquet file


buffer = io.BytesIO()  #Creating an buffer
object = resource.Object(bucket_name,product_dates[1][0])
object.download_fileobj(buffer)

product_file_1 = pd.read_parquet(buffer) #Reading the previous product parquet file


# buffer = io.BytesIO()
# object = resource.Object(bucket_name,product_dates[2][0])
# object.download_fileobj(buffer)

# product_file_0 = pd.read_parquet(buffer)
# product_file_0



In [5]:
data = {
    'Agent_name': [],
    'Issue': [],
    'Specific_column': [],
    'Description': [],
    'Missing_categories': [],
    'URL': []
}

df = pd.DataFrame(data)
df

,Agent_name,Issue,Specific_column,Description,Missing_categories,URL


##### Run ID and Run Date for product file

In [6]:
run_freq_product_1 = product_file_1["RunID"][0]  #Getting Run ID of previous file
run_date_product_1 = product_file_1["RunDate"][0] #Getting Run Date of previous file

run_freq_product_2 = product_file_2["RunID"][0]  #Latest File
run_date_product_2 = product_file_2["RunDate"][0] #Getting Run Date of Latest file
run_freq_product_1

1893

##### Run ID and Run Date for Category file

In [7]:
run_freq_category_1 = category_file_1["RunID"][0]  #Getting Run ID of previous file
run_date_category_1 = category_file_1["RunDate"][0] #Getting Run Date of previous file

run_freq_category_2 = category_file_2["RunID"][0]  #Latest File
run_date_category_2 = category_file_2["RunDate"][0] #Getting Run Date of Latest file

##### Check for the incremented run date & run id for product file

In [8]:
def checkIncrement(latest,previous) : #Checking the latest file has greater value or not.
    if(latest < previous) : #Checking the condition weather latest data is greater than previous or not
        return True
    return False

In [9]:
if(checkIncrement(run_freq_product_2,run_freq_product_1)): #Calling the checkIncrement function and checking weather the latest freq greather than the previous file
    print("The Latest run id less than the previous run id")
    specific_column = product_file_2.columns[product_file_2.columns == 'RunID'][0]
    description = "Previous RunId :" + str(run_freq_product_1) + "Latest RunId :" + str(run_freq_product_2)
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Increment', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[0])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("Run Id is in increasing order")


Run Id is in increasing order


In [10]:
if(checkIncrement(run_date_product_2,run_date_product_1)): #Calling the checkIncrement function and checking weather the latest Runid greather than the previous file
    print("The Latest Run Date less than the previous run id")
    specific_column = product_file_2.columns[product_file_2.columns == 'RunDate'][0]
    description = "Previous RunDate :" + str(run_date_product_1) + "Latest RunDate :" + str(run_date_product_2)
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Increment', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[1])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("Run date is in increasing order")

Run date is in increasing order


In [11]:
product_file_1.head()

,RunDate,CategoryId,SKU,UPC,ProductId,ProductName,Price,SalePrice,RunID
0,2023-10-04 05:58:26,Men-Training,1361588,,1361588,Men's HeatGear® ¾ Leggings,26.9700,None,1893
1,2023-10-04 07:13:11,Kids-Boys-Pants & Leggings,5112715,,5112715,Little Boys' UA Halftone Reaper Pieced Joggers,32.0000,28.9700,1893
2,2023-10-04 07:13:11,Shoes-Featured-New Arrivals,3026704,,3026704,Boys' Pre-School UA Flash Running Shoes,45.0000,None,1893
3,2023-10-04 07:13:11,Kids-Running & Training,1363284,,1363284,Boys' UA Tech™ 2.0 Short Sleeve,20.0000,14.9900,1893
4,2023-10-04 06:39:21,Kids-Featured-Best Sellers,1362489,,1362489,Boys' UA Match Play Shorts,45.0000,40.9700,1893


###### In Category file, The categories should be matched with product file

In [13]:
category_data = category_file_2[category_file_2['lvl4'] == ""] #Getting the rows in which the lvl4 and lvl5 are empty
category_data = category_file_1[category_file_1['lvl3'] != ""]
category_data = category_file_1[category_file_1['lvl2'] != ""]
categories = category_data['CategoryName'].unique()  #Making the CategoryName Unique in category file
products = product_file_2["CategoryId"].unique()     #Making the CategoryId Unique in product file
pattern = r'-(?![^(]*\))' 

products = np.array([re.split(pattern, item)[-1] for item in products]) #Getting the last category from product file

result = np.array([any(element in keyword for keyword in categories) for element in products]) #Checking weather the last category in product file matching with categories in categories file
if result.all(): #Checking weather all the values are True or not
    print("The Categories in Category File is matching with product file")
    print(products)
else :
    category_data = category_file_1[category_file_1['lvl4'] == ""] #Getting the rows in which the lvl4 and lvl5 are empty
    category_data = category_file_1[category_file_1['lvl3'] != ""]
    category_data = category_file_1[category_file_1['lvl2'] != ""]
    categories = category_data['CategoryName'].unique()  #Making the CategoryName Unique in category file
    products = product_file_2["CategoryId"].unique()     #Making the CategoryId Unique in product file
    pattern = r'-(?![^(]*\))' 
    products = np.array([re.split(pattern, item)[-1] for item in products]) #Getting the last category from product file

    result = np.array([any(element in keyword for keyword in categories) for element in products]) #Checking weather the last category in product file matching with categories in categories file
    
    if result.all(): #Checking weather all the values are True or not
        print("The Categories in Category File is matching with product file")
    else :
        print("The Categories in Category File is not matching with product file")
        index = np.where(result == False)
        missing_categoies = str(products[index])
        specific_column = product_file_2.columns[product_file_2.columns == 'CategoryId'][0]
        description = "The Categories in product file is missing in category file"
        new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Category Missing', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": missing_categoies,'URL': ''},index=[2])
        df = pd.concat([df, new_data], ignore_index=True)


The Categories in Category File is not matching with product file


In [14]:
def checkNull(data):  #Function for checking the null values in particular column of dataframe
    if(data.isnull().sum() or data.all() == "None"):  
        return True
    return False

##### Checking in Category file weather the particular columns contains null or not

In [15]:
if(checkNull(product_file_2['CategoryId'])):  #Passing Category Id column in product file to check weather it contains null entries or not
    print("CategoryId contains Null values")
    specific_column = product_file_2.columns[product_file_2.columns == 'CategoryId'][0]
    description = "Category Id's is missing in product file"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[3])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("CategoryId does not contain null values")

CategoryId does not contain null values


In [16]:
if(checkNull(product_file_2['SKU'])):  #Passing SKU column in product file to check weather it contains null entries or not
    print("SKU contains Null values")
    specific_column = product_file_2.columns[product_file_2.columns == 'SKU'][0]
    description = "SKU values are missing in product file"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[4])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("SKU does not contain null values")

SKU does not contain null values


In [17]:
if(checkNull(product_file_2['ProductName'])):  #Passing ProductName column in product file to check weather it contains null entries or not
    print("ProductName contains Null values")
    specific_column = product_file_2.columns[product_file_2.columns == 'ProductName'][0]
    description = "ProductName values are missing in product file"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[5])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("ProductName does not contain null values")

ProductName does not contain null values


In [18]:
average_file_size = (product_file_2.shape[0] + product_file_1.shape[0])/2 #Average file count of latest 3 product files
percentage_count = ((average_file_size - product_file_2.shape[0])/(average_file_size))*100 #Getting the percentage count
if percentage_count > 5 or percentage_count < -5 :
    print("There is an issue with the count")
    description = "Previous Data Count:" + str(product_file_1.shape[0]) + "Latest Data count" + str(product_file_2.shape[0]) + "Percentage count:" + str(percentage_count)
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Data Count', 'Specific_column': '', 'Description': description ,"Missing_categories": '','URL': ''},index=[6])
    df = pd.concat([df, new_data], ignore_index=True)
else :
    print("Data count is fine")
    
average_file_size

Data count is fine


22184.5

In [19]:
#Getting the duplicate count on the basis of unique id
duplicates = product_file_2[product_file_2.duplicated(subset=['CategoryId', 'ProductId'])] 
if(duplicates.shape[0] > 0) :
    print(duplicates.shape[0],"duplicates were found")
    specific_column = product_file_2.columns[product_file_2.columns == 'ProductId'][0]
    description = str(duplicates['ProductId']) + "Found duplicates"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Duplicates', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[7])
    df = pd.concat([df, new_data], ignore_index=True)
else :
    print("No duplicates were found")

No duplicates were found


In [20]:
#total count on the basis of unique id
total_count = (product_file_2['CategoryId'].astype(str) + '-' + product_file_2['ProductId'].astype(str))
print("total count is",total_count.shape[0])

total count is 22372


In [21]:
#Passing RunID column in Category file to check weather it contains null entries or not
if(checkNull(category_file_2['RunID'])):  
    print("RunID contains Null values")
    specific_column = category_file_2.columns[category_file_2.columns == 'RunID'][0]
    description = "RunID contains null vales"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[8])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("RunID does not contain null values")

RunID does not contain null values


In [22]:
if(checkNull(category_file_2['RunDate'])):  #Passing RunDate column in Category file to check weather it contains null entries or not
    print("RunDate contains Null values")
    specific_column = category_file_2.columns[category_file_2.columns == 'RunDate'][0]
    description = "RunDate contains null vales"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[9])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("RunDate does not contain null values")

RunDate does not contain null values


In [23]:
if(checkNull(category_file_2['Url'])):  #Passing Url column in Category file to check weather it contains null entries or not
    print("Url contains Null values")
    specific_column = category_file_2.columns[category_file_2.columns == 'Url'][0]
    description = "Url contains null vales"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[10])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("Url does not contain null values")

Url does not contain null values


In [24]:
if(checkNull(category_file_2['CategoryID'])):  #Passing CategoryID column in Category file to check weather it contains null entries or not
    print("CategoryID contains Null values")
    specific_column = category_file_2.columns[category_file_2.columns == 'CategoryID'][0]
    description = "CategoryID contains null vales"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[11])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("CategoryID does not contain null values")

CategoryID does not contain null values


In [25]:
if(checkNull(category_file_2['CategoryLevel'])):  #Passing CategoryLevel column in Category file to check weather it contains null entries or not
    print("CategoryLevel contains Null values")
    specific_column = category_file_2.columns[category_file_2.columns == 'CategoryLevel'][0]
    description = "CategoryLevel contains null vales"
    new_data = pd.DataFrame({'Agent_name': 'underarmour', 'Issue': 'Check Null', 'Specific_column': specific_column, 'Description': description ,"Missing_categories": '','URL': ''},index=[12])
    df = pd.concat([df, new_data], ignore_index=True)
else:
    print("CategoryLevel does not contain null values")

CategoryLevel does not contain null values


In [26]:
product_file = product_file_2[product_file_2['SalePrice'] != product_file_2['SalePrice'].isnull()]
product_file

,RunDate,CategoryId,SKU,UPC,ProductId,ProductName,Price,SalePrice,RunID
0,2023-10-05 05:27:37,New & Featured-Featured-UA Fleece Shop,1377446,,1377446,Women's Project Rock Heavyweight Terry Full-Zip,95.0000,None,1894
1,2023-10-05 05:58:16,Men-Shoes,3022707,,3022707,Men's UA Ignite III Sandals,35.0000,31.9700,1894
2,2023-10-05 07:15:01,Kids-Running & Training,5119951,,5119951,Little Boys' UA Fader Block Logo Long Sleeve &...,40.0000,None,1894
3,2023-10-05 08:13:03,Outlet-Girls-Shorts,1377024,,1377024,Girls' UA Play Up Big Logo Graphic Shorts,25.0000,18.9700,1894
4,2023-10-05 05:27:37,New & Featured-Featured-UA Fleece Shop,5120630,,5120630,Men's UA Tech™ Terry Gameday Collegiate ¼ Zip,75.0000,None,1894
...,...,...,...,...,...,...,...,...,...
22367,2023-10-05 08:13:03,Outlet-Boys-Shirts & Tops,1378939,,1378939,Kids' UA Pride Short Sleeve,30.0000,24.9700,1894
22368,2023-10-05 05:58:16,Men-UA HeatGear®: Keeps You Cool,1376427,,1376427,Men's UA Gameday Armour Pro 5-Pad Top,75.0000,None,1894
22369,2023-10-05 05:27:37,New & Featured-Featured-Sportstyle,1373101,,1373101,Women's UA Halftime Cuff Beanie,20.9700,None,1894
22370,2023-10-05 07:15:01,Kids-Big Kids (8-20),1364980,,1364980,Boys' UA Command Warm-Up Pants,54.9700,None,1894


In [27]:
product_file_2.head(3)

,RunDate,CategoryId,SKU,UPC,ProductId,ProductName,Price,SalePrice,RunID
0,2023-10-05 05:27:37,New & Featured-Featured-UA Fleece Shop,1377446,,1377446,Women's Project Rock Heavyweight Terry Full-Zip,95.0000,None,1894
1,2023-10-05 05:58:16,Men-Shoes,3022707,,3022707,Men's UA Ignite III Sandals,35.0000,31.9700,1894
2,2023-10-05 07:15:01,Kids-Running & Training,5119951,,5119951,Little Boys' UA Fader Block Logo Long Sleeve &...,40.0000,None,1894


In [28]:
category_id_groupby_1 = product_file_2["CategoryId"].value_counts()
category_id_groupby_1 = product_file_1["CategoryId"].value_counts()
category_id_groupby_1
# category_id_groupby_1.values == category_id_groupby_2.values

Kids-Running & Training                992
Kids-Big Kids (8-20)                   795
Men-Training                           777
Women-Training                         721
Men-Shirts & Tops                      638
                                      ... 
New & Featured-Gift Guide-Gift Card      2
Men-UA SlipSpeed™                        2
Women-UA SlipSpeed™                      2
Shoes-Featured-UA SlipSpeed™             2
Outlet-Men-Underwear                     1
Name: CategoryId, Length: 248, dtype: int64

In [29]:
Previous_data = product_file_1.shape[0]
Current_data = product_file_2.shape[0]
Detail_count_difference = Current_data - Previous_data

In [30]:
# Email configuration
smtp_server = "smtp.gmail.com"
smtp_port = 587
sender_email = "sriramojuabhishek264@gmail.com"
sender_password = "pzwr kphf sshp sopc"
recipient_email = "abhishek.sriramoju@sequentum.com"


underarmour_report = [[Previous_data,Current_data,Detail_count_difference, percentage_count]]
# # underarmour_report = []


table_header1 = ["Previous_Day_Count",
                     "Today_Count",
                     "Count_Difference",
                     "Percentage_Difference(%)"
                     ]

# # data into a Pandas DataFrame
extracted_data = pd.DataFrame(underarmour_report, columns=table_header1)

#styled_extracted_data = extracted_data.style.hide_index()

#styled_extracted_data_new = extracted_data.format({'Percentage_Difference(%)': '{:.2f}%'})

issue_table = build_table(df, "blue_light")

issue_table_analysis = build_table(extracted_data, "blue_light")

# append in email_body and return it
email_body = "<strong><h3>\n\nPlease check the below table for data issues.</h3></strong>\n"
email_body += issue_table
email_body += issue_table_analysis

# Create the email message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = recipient_email
msg['Subject'] = 'Table in Email'

# Attach the HTML message to the email
#msg.attach(MIMEText(table_html, 'html'))

attachment = MIMEText(df.to_csv(index=False), 'csv')
analysis_attachment = MIMEText(extracted_data.to_csv(index=False), 'csv')


attachment.add_header('Content-Disposition', 'attachment', filename='underarmour_report.csv')
msg.attach(attachment)
msg.attach(MIMEText(email_body,'html'))


# Connect to the SMTP server and send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipient_email, msg.as_string())

print("Email sent successfully.")

Email sent successfully.


In [55]:
underarmour_report

[[21997, 22372, 375, -0.8451847010299985]]